```
Langchain 을 활용한 모델 사용, 비용 모니터링 및 캐싱 전략
    GPT-4o-mini GPT-3.5-Turbo 비용이 60% 저렴
    LangChain v0.3x 부터 openAi가 별도 패키지로 분리 필요 패키지를 설치 langchain-openai 필요
    토큰사용량 추적, 캐싱을 위한 langchain-community도 별도설치
    환경변수 변수 관리 패키지 python-dotenv
```

In [1]:
%pip install langchain-openai langchain-community python-dotenv openai

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 46.1 MB/s eta 0:00:00

   ---------------------------- ----------- 5/7 [dataclasses-json]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   ---------------------------------- ----- 6/7 [langchain-community]
   -------------

In [ ]:
from dotenv import load_dotenv
import os

# 현재 작업 디렉토리에 있는 .env 파일 경로 지정
env_path = os.path.join(os.getcwd(), '.env')
load_dotenv(env_path, override=True)  # 덮어쓰기 옵션 추가

print("현재 작업 디렉토리:", os.getcwd())
print("불러온 OPENAI_API_KEY:", repr(os.getenv("OPENAI_API_KEY")))


In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7) # dotenv를 하면 자동으로 OPENAI_API_KEY
prompt = 'LangChain에 대해 한 문장으로 설명해줘'
result = llm.invoke(prompt)
result.content

'LangChain은 다양한 언어 모델과 데이터 소스를 통합하여 자연어 처리 애플리케이션을 쉽게 구축할 수 있도록 돕는 프레임워크입니다.'

In [6]:
# 사용량
result.usage_metadata

{'input_tokens': 18,
 'output_tokens': 37,
 'total_tokens': 55,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [7]:
# 콜백함수를 통한 누적 토큰 추적(get_openai_callback)
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫번째 호출
    res1 = llm.invoke('서울의 오늘 날씨는 어떤지 알려줘?')
    print('응답1', res1.content[:10],'...')
    # 두번째 호출
    res2 = llm.invoke('파이썬으로 랭체인 사용법을 알려줘')
    print('응답2', res2.content[:10],'...')

# 누적 토큰 사용량 출력 콜백 cb에는 블록 내 전체 토큰 사용량이 누적됨
# 총 토큰수
print('총 토큰수:', cb.total_tokens)
# 프롬프트 토큰수
print('프롬프트 토큰수:', cb.prompt_tokens)
# 응답 토큰수
print('응답 토큰수:', cb.completion_tokens)
# 비용 계산
print('비용(USD):', cb.total_cost)


응답1 죄송하지만, 현재  ...
응답2 랭체인(Chain  ...
총 토큰수: 695
프롬프트 토큰수: 39
응답 토큰수: 656
비용(USD): 0.00039945


In [8]:
# LangChain의 LLM 응답캐싱 (InMemory Cache, SQLite Cache)
# 동일한 질문은 저장해 뒀다가 응답에 사용
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCache를 LLM 캐시로 설정
set_llm_cache(InMemoryCache())

In [9]:
# 캐시 사용 전후를 비교, 같은 질문을 두번 호출
query = "재미있는 유머 하나 알려줘"
# 첫 번째 호출(캐시에 없으면 api 호출 발생)
result1 = llm.invoke(query)
print("응답1:", {result1.content})

# 두 번째 호출(동일한 query, 캐시를 확인하고 동일 질문이면 api 미호출)
result2 = llm.invoke(query)
print("응답2:", {result2.content})

응답1: {'물고기가 컴퓨터를 사용하면 어떤 일이 벌어질까요?\n\n바로 "웹 서핑"을 하게 되죠! 🐟💻'}
응답2: {'물고기가 컴퓨터를 사용하면 어떤 일이 벌어질까요?\n\n바로 "웹 서핑"을 하게 되죠! 🐟💻'}


In [11]:
# 실행시간 측정
import time
# 첫 번째 호출 시간
query = "점심메뉴 추천해줘"
start = time.time()
llm.invoke(query)
end = time.time()
print("첫 번째 호출 시간:", end - start)
# 두 번째 호출 시간
start = time.time()
llm.invoke(query)
end = time.time()
print("두 번째 호출 시간:", end - start)    

첫 번째 호출 시간: 4.935808181762695
두 번째 호출 시간: 0.0005819797515869141


In [16]:
# SQLite Cache 사용
import os
from langchain.cache import SQLiteCache

# 기존 캐시 DB 삭제(.langchain.db)
#if os.path.exists('.langchain.db'):
#    os.remove('.langchain.db')
cache = SQLiteCache(database_path=".langchain.db")
set_llm_cache(cache)

def close_cache_connection(cache):
    if hasattr(cache, "conn"):
        cache.conn.close()
        print("캐시 DB 연결 종료 완료")
    else:
        print("캐시 객체에 conn 속성이 없습니다.")




# SQLite 캐시 설정(지정한 경로의 DB 파일을 생성 / 사용)
set_llm_cache(SQLiteCache(database_path = '.langchain.db')) # langchain.db있으면 사용 없으면 생성

# 동일한 query를 두 번 호출해서 결과와 시간을 비교
query = "썰렁한 유머 하나 알려줘"
# 첫 번째 호출(캐시에 없으면 api 호출 발생)
start = time.time()
result1 = llm.invoke(query)
end = time.time()   
print("응답1:", {result1.content})
print("첫 번째 호출 시간:", end - start)
# 두 번째 호출(동일한 query, 캐시를 확인하고 동일 질문이면 api 미호출)
start = time.time()
result2 = llm.invoke(query)
end = time.time()
print("응답2:", {result2.content})
print("두 번째 호출 시간:", end - start)
# 캐시 DB 파일 확인
if os.path.exists('.langchain.db'):
    print("SQLite 캐시 DB 파일이 생성되었습니다: .langchain.db")

close_cache_connection(cache)

응답1: {'왜 바나나는 혼자 다니지 않을까요?\n\n항상 "껍질"이 두렵거든요! 🍌😄'}
첫 번째 호출 시간: 0.003416299819946289
응답2: {'왜 바나나는 혼자 다니지 않을까요?\n\n항상 "껍질"이 두렵거든요! 🍌😄'}
두 번째 호출 시간: 0.0011670589447021484
SQLite 캐시 DB 파일이 생성되었습니다: .langchain.db
캐시 객체에 conn 속성이 없습니다.
